In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re
import pickle

In [2]:
df_formation = pd.read_csv("../data/matches/formations_1718_premier_league.csv")
df_lineup = pd.read_csv("../data/matches/lineups_1718_premier_league.csv")

In [3]:
df_formation.head()

,gameid,team,formation,players
0,22342,Arsenal,3-4-2-1,33;16 18 31;24 35 29 15;11 23;9
1,22342,Leicester City,4-4-1-1,1;2 5 15 28;26 22 25 11;20;9
2,22343,Brighton and Hove Albion,4-4-2,1;2 22 5 29;20 6 24 37;13 10
3,22343,Manchester City,3-1-4-2,31;4 5 30;25;2 17 21 3;10 33
4,22344,Chelsea,3-4-2-1,13;2 30 24;28 4 7 3;22 38;23


In [4]:
df_lineup.head()

,gameid,type,team,name,number
0,22342,home,Arsenal,Petr Cech,33
1,22342,home,Arsenal,Sead Kolasinac,31
2,22342,home,Arsenal,Rob Holding,16
3,22342,home,Arsenal,Nacho Monreal,18
4,22342,home,Arsenal,Mohamed Elneny,35


In [15]:
def group_rows(row):
    team_group = list(row['team'])
    formation_group = list(row['formation'])
    players_group = list(row['players'])
    return pd.Series([team_group,formation_group,players_group])

In [34]:
df_formation_match = pd.DataFrame(df_formation.groupby('gameid').apply(lambda x: group_rows(x)))

In [35]:
df_formation_match.reset_index(inplace=True)

In [36]:
df_formation_match.rename(columns = {0:'team',1:'formation',2:'players'},inplace=True)

In [37]:
df_formation_match.head()

,gameid,team,formation,players
0,22342,"[Arsenal, Leicester City]","[3-4-2-1, 4-4-1-1]","[33;16 18 31;24 35 29 15;11 23;9, 1;2 5 15 28;..."
1,22343,"[Brighton and Hove Albion, Manchester City]","[4-4-2, 3-1-4-2]","[1;2 22 5 29;20 6 24 37;13 10, 31;4 5 30;25;2 ..."
2,22344,"[Chelsea, Burnley]","[3-4-2-1, 4-4-1-1]","[13;2 30 24;28 4 7 3;22 38;23, 1;2 5 6 23;17 1..."
3,22345,"[Crystal Palace, Huddersfield Town]","[3-4-2-1, 4-2-3-1]","[13;24 6 44;2 4 42 3;8 11;17, 1;2 25 26 15;10 ..."
4,22346,"[Everton, Stoke City]","[3-4-2-1, 3-4-2-1]","[1;4 5 6;29 2 17 3;10 20;9, 1;6 17 20;18 4 24 ..."


In [38]:
def split_cols(row):
    return pd.Series([row['team'][0],row['team'][1],row['formation'][0],row['formation'][1],row['players'][0],row['players'][1]])

In [39]:
df_formation_match[['home_team','away_team','home_formation','away_formation','home_players','away_players']] = df_formation_match.apply(lambda x: split_cols(x),axis=1)

In [40]:
df_formation_match.head()

,gameid,team,formation,players,home_team,away_team,home_formation,away_formation,home_players,away_players
0,22342,"[Arsenal, Leicester City]","[3-4-2-1, 4-4-1-1]","[33;16 18 31;24 35 29 15;11 23;9, 1;2 5 15 28;...",Arsenal,Leicester City,3-4-2-1,4-4-1-1,33;16 18 31;24 35 29 15;11 23;9,1;2 5 15 28;26 22 25 11;20;9
1,22343,"[Brighton and Hove Albion, Manchester City]","[4-4-2, 3-1-4-2]","[1;2 22 5 29;20 6 24 37;13 10, 31;4 5 30;25;2 ...",Brighton and Hove Albion,Manchester City,4-4-2,3-1-4-2,1;2 22 5 29;20 6 24 37;13 10,31;4 5 30;25;2 17 21 3;10 33
2,22344,"[Chelsea, Burnley]","[3-4-2-1, 4-4-1-1]","[13;2 30 24;28 4 7 3;22 38;23, 1;2 5 6 23;17 1...",Chelsea,Burnley,3-4-2-1,4-4-1-1,13;2 30 24;28 4 7 3;22 38;23,1;2 5 6 23;17 16 4 12;13;9
3,22345,"[Crystal Palace, Huddersfield Town]","[3-4-2-1, 4-2-3-1]","[13;24 6 44;2 4 42 3;8 11;17, 1;2 25 26 15;10 ...",Crystal Palace,Huddersfield Town,3-4-2-1,4-2-3-1,13;24 6 44;2 4 42 3;8 11;17,1;2 25 26 15;10 8;9 45 22;24
4,22346,"[Everton, Stoke City]","[3-4-2-1, 3-4-2-1]","[1;4 5 6;29 2 17 3;10 20;9, 1;6 17 20;18 4 24 ...",Everton,Stoke City,3-4-2-1,3-4-2-1,1;4 5 6;29 2 17 3;10 20;9,1;6 17 20;18 4 24 3;22 27;9


In [41]:
df_formation_match.drop(['team','formation','players'],axis=1,inplace=True)

In [42]:
df_formation_match.head()

,gameid,home_team,away_team,home_formation,away_formation,home_players,away_players
0,22342,Arsenal,Leicester City,3-4-2-1,4-4-1-1,33;16 18 31;24 35 29 15;11 23;9,1;2 5 15 28;26 22 25 11;20;9
1,22343,Brighton and Hove Albion,Manchester City,4-4-2,3-1-4-2,1;2 22 5 29;20 6 24 37;13 10,31;4 5 30;25;2 17 21 3;10 33
2,22344,Chelsea,Burnley,3-4-2-1,4-4-1-1,13;2 30 24;28 4 7 3;22 38;23,1;2 5 6 23;17 16 4 12;13;9
3,22345,Crystal Palace,Huddersfield Town,3-4-2-1,4-2-3-1,13;24 6 44;2 4 42 3;8 11;17,1;2 25 26 15;10 8;9 45 22;24
4,22346,Everton,Stoke City,3-4-2-1,3-4-2-1,1;4 5 6;29 2 17 3;10 20;9,1;6 17 20;18 4 24 3;22 27;9


In [43]:
list(map(int,df_formation_match['home_players'][0].split(';')[1].split(' ')))

[16, 18, 31]

In [44]:
df_lineup['key'] = df_lineup.apply(lambda x: str(x['gameid']) + x['team'] + str(x['number']), axis=1)

In [45]:
df_lineup_dict = df_lineup[['key','name']].set_index('key').to_dict()

In [46]:
def get_defenders(row,dictionary):
    home_def_jersey = list(map(int,row['home_players'].split(';')[1].split(' ')))
    away_def_jersey = list(map(int,row['away_players'].split(';')[1].split(' ')))
    home_def_names = list()
    away_def_names = list()
    for val in home_def_jersey:
        key = str(row['gameid'])+row['home_team']+str(val)
        player_name = dictionary['name'][key]
        player_name_clean = re.sub('[^a-zA-z]','',unidecode(player_name.strip()))
        home_def_names.append(player_name_clean)
    for val in away_def_jersey:
        key = str(row['gameid'])+row['away_team']+str(val)
        player_name = dictionary['name'][key]
        player_name_clean = re.sub('[^a-zA-z]','',unidecode(player_name.strip()))
        away_def_names.append(player_name_clean)
    return pd.Series([home_def_names,away_def_names])
    

In [47]:
df_formation_match[['home_team_defense','away_team_defense']] = df_formation_match.apply(lambda x: get_defenders(x,df_lineup_dict),axis=1)

In [48]:
df_formation_match.loc[(df_formation_match['home_team']=='Manchester United')&(df_formation_match['away_team']=='Newcastle United')]

,gameid,home_team,away_team,home_formation,away_formation,home_players,away_players,home_team_defense,away_team_defense
117,22459,Manchester United,Newcastle United,4-2-3-1,4-4-2,1;25 2 12 18;6 31;19 8 11;9,1;22 20 2 19;7 14 8 11;9 21,"[LuisAntonioValencia, VictorLindelof, ChrisSma...","[DeAndreYedlin, FlorianLejeune, CiaranClark, J..."


In [49]:
df_formation_match.drop(['home_formation','away_formation','home_players','away_players'],axis=1,inplace=True)

In [50]:
df_formation_match.rename(columns={'gameid':'match_id'},inplace=True)

In [51]:
df_formation_match.head()

,match_id,home_team,away_team,home_team_defense,away_team_defense
0,22342,Arsenal,Leicester City,"[RobHolding, NachoMonreal, SeadKolasinac]","[DannySimpson, WesMorgan, HarryMaguire, Christ..."
1,22343,Brighton and Hove Albion,Manchester City,"[Bruno, ShaneDuffy, LewisDunk, MarkusSuttner]","[VincentKompany, JohnStones, NicolasOtamendi]"
2,22344,Chelsea,Burnley,"[AntonioRudiger, DavidLuiz, GaryCahill]","[MatthewLowton, JamesTarkowski, BenMee, Stephe..."
3,22345,Crystal Palace,Huddersfield Town,"[TimothyFosuMensah, ScottDann, JairoRiedewald]","[TommySmith, Zanka, ChristopherSchindler, Chri..."
4,22346,Everton,Stoke City,"[MichaelKeane, AshleyWilliams, PhilJagielka]","[KurtZouma, RyanShawcross, GeoffCameron]"


In [52]:
df_formation_match.to_csv("../data/matches/PLopensource.csv")